In [1]:
from tg.grammar_ru.ml.corpus import CorpusReader
from tg.grammar_ru.common import Loc
from pathlib import Path

base_path = Loc.corpus_path/'example.base.zip'
reader = CorpusReader(Path(base_path))

In [3]:
from tg.grammar_ru.algorithms import AntecedentCandidatesAlgorithm

alg = AntecedentCandidatesAlgorithm(reader.get_bundles(), 7)

c:\users\alexandra\desktop\grammar_ru\venv\lib\site-packages\slovnet\model\emb.py:46: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_numpy.cpp:178.)
  torch.from_numpy(navec.pq.indexes),


Конечно, никто никогда в мире не писал замену имён собственных, поэтому напишем:

In [4]:
alg.pronoun_replacer

{'академия': 'женщина',
 'алевтина': 'женщина',
 'амелия': 'женщина',
 'андрий': 'мужчина',
 'аннегрета': 'женщина',
 'бэнд': 'мужчина',
 'ванко': 'мужчина',
 'верин': 'мужчина',
 'викторович': 'женщина',
 'глас': 'мужчина',
 'густав': 'мужчина',
 'денежкин': 'мужчина',
 'исповедание': 'существо',
 'кареолан': 'мужчина',
 'карл': 'мужчина',
 'каролина': 'женщина',
 'каспер': 'мужчина',
 'кик': 'мужчина',
 'кит': 'мужчина',
 'лилиана': 'женщина',
 'лилия': 'женщина',
 'марек': 'мужчина',
 'мартин': 'мужчина',
 'матвей': 'мужчина',
 'михал': 'мужчина',
 'никита': 'мужчина',
 'оливер': 'мужчина',
 'ополе': 'существо',
 'патрик': 'мужчина',
 'росинтеллектсистема': 'женщина',
 'снегур': 'мужчина',
 'стефанос': 'мужчина',
 'творец': 'мужчина',
 'тор': 'мужчина',
 'торам': 'мужчина',
 'франтишек': 'мужчина',
 'фредерик': 'мужчина',
 'ханна': 'женщина',
 'ханне': 'женщина',
 'чисовица': 'женщина',
 'ярина': 'женщина',
 'ярином': 'мужчина'}

In [5]:
db = reader.get_bundles().first()

In [6]:
# то, что, вероятно, было выделением имён собственных в slovnet:
from tg.grammar_ru.ml.features import SlovnetFeaturizer

featurizer = SlovnetFeaturizer()
featurizer.featurize(db)
db.src[db.slovnet.POS == 'PROPN'].word.head(15)

0              Лилия
63          Закричав
65             Лилия
118           Голова
142            Лилия
152         Выманили
208            Лилия
220    Прикоснувшись
275            Лилия
333        Чисовицах
346        Девятерых
355           Лилией
370         Академии
372         Каролина
402            Лилия
Name: word, dtype: object

In [7]:
from tg.grammar_ru.ml.features import PyMorphyFeaturizer

featurizer = PyMorphyFeaturizer()
featurizer.featurize(db)
db.pymorphy['normal_form'] = db.pymorphy['normal_form'].apply(lambda x: alg.pronoun_replacer[x] if x in alg.pronoun_replacer else x)
db.pymorphy.head()

,normal_form,alternatives,score,delta_score,POS,animacy,gender,number,case,aspect,transitivity,person,tense,mood,voice,involvement
word_id,,,,,,,,,,,,,,,,
0,женщина,2,0.666666,0.333333,NOUN,anim,femn,sing,nomn,NaN,NaN,NaN,NaN,NaN,NaN,None
1,очнуться,1,1.000000,1.000000,VERB,NaN,femn,sing,NaN,perf,intr,NaN,past,indc,NaN,None
2,от,1,1.000000,1.000000,PREP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
3,суетливый,3,0.333333,0.000000,ADJF,NaN,masc,sing,gent,NaN,NaN,NaN,NaN,NaN,NaN,None
4,",",1,1.000000,1.000000,NONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


In [8]:
candidates_df = alg.get_candidates(db)
with_parents_df = alg.get_pronoun_parent(db, candidates_df)
with_parents_df

,pronoun_word_id,candidate_word_id,candidate_distance,pronoun_parent_id
0,10,0,0,9
1,18,0,1,19
2,18,10,0,19
3,30,0,2,31
4,30,10,1,31
...,...,...,...,...
303,921,907,4,922
304,921,914,3,922
305,921,917,2,922
306,921,918,1,922


In [38]:
from tg.grammar_ru.ml.features import NavecFeaturizer

product_df = with_parents_df.merge(db.pymorphy.normal_form.rename('pronoun_parent_norm'), left_on='pronoun_parent_id', right_index=True, how='left')
product_df = product_df.merge(db.pymorphy.normal_form.rename('candidate_word_norm'), left_on='candidate_word_id', right_index=True, how='left')
featurizer = NavecFeaturizer()
product_df['product'] = featurizer.get_glove_prod(product_df, 'pronoun_parent_norm', 'candidate_word_norm')
product_df

,pronoun_word_id,candidate_word_id,candidate_distance,pronoun_parent_id,pronoun_parent_norm,candidate_word_norm,product
0,10,0,0,9,преследовать,женщина,2.660135
1,18,0,1,19,гнаться,женщина,-1.817963
2,18,10,0,19,гнаться,она,-0.213960
3,30,0,2,31,оставаться,женщина,7.911864
4,30,10,1,31,оставаться,она,8.539292
...,...,...,...,...,...,...,...
303,921,907,4,922,кубарем,жертва,-0.980053
304,921,914,3,922,кубарем,судьба,-1.951545
305,921,917,2,922,кубарем,она,-0.423255
306,921,918,1,922,кубарем,роль,-1.972080


In [25]:
# Можно ли улучшить выбор кандидатов, если отбрасывать детей родителя местоимения:
pronoun_parent_ids = db.slovnet[db.slovnet.index.isin(candidates_df['pronoun_word_id'])]['syntax_parent_id']
candidate_parent_ids = db.slovnet[db.slovnet.index.isin(candidates_df['candidate_word_id'])]['syntax_parent_id']
parents_compare = (candidates_df
 .merge(pronoun_parent_ids.to_frame().reset_index(), left_on='pronoun_word_id', right_on='word_id')
 .merge(candidate_parent_ids.to_frame().reset_index(), left_on='candidate_word_id', right_on='word_id'))
parents_compare[parents_compare['syntax_parent_id_x'] == parents_compare['syntax_parent_id_y']]

,pronoun_word_id,candidate_word_id,candidate_distance,word_id_x,syntax_parent_id_x,word_id_y,syntax_parent_id_y
303,917,914,0,917,916,914,916


In [26]:
product_df[product_df['pronoun_word_id'] == 917]

,pronoun_word_id,candidate_word_id,candidate_distance,pronoun_parent_id,pronoun_parent_norm,candidate_word_norm,product
294,917,878,6,916,уготовить,паника,-2.770621
295,917,884,5,916,уготовить,женщина,-5.830828
296,917,894,4,916,уготовить,лестница,-2.799802
297,917,896,3,916,уготовить,женщина,-5.830828
298,917,905,2,916,уготовить,она,-5.512835
299,917,907,1,916,уготовить,жертва,-0.058948
300,917,914,0,916,уготовить,судьба,3.859842


На первый DataBundle всего один случай ошибки при совпадающем у местоимения и кандидата в антецеденты родителе, осталось бы ошибкой, вероятно, пока можно не трогать.

In [138]:
def get_big_deviation_only(df):
    product_groups = df.drop_duplicates(['pronoun_parent_norm','candidate_word_norm'], keep='last').groupby(['pronoun_word_id'])['product']
    maxes = product_groups.max()#.dropna().astype(int).dropna().astype(int)
    means = product_groups.mean()
    stds = product_groups.std()
    max_indices = product_groups.idxmax().dropna().astype(int)
    best_in_group = df.loc[max_indices]
    temp = ((product_groups.count() < 4) | (stds < maxes - means)).to_frame()
    return best_in_group[best_in_group.pronoun_word_id.isin(temp[temp['product']].index)]

get_big_deviation_only(product_df)

,pronoun_word_id,candidate_word_id,candidate_distance,pronoun_parent_id,pronoun_parent_norm,candidate_word_norm,product
0,10,0,0,9,преследовать,женщина,2.660135
2,18,10,0,19,гнаться,она,-0.213960
5,30,18,0,31,оставаться,она,8.539292
15,53,30,0,52,подвести,она,1.963640
20,56,53,0,57,упасть,она,7.820927
23,83,56,4,84,увидеть,она,10.808409
30,113,95,4,114,напугать,девушка,5.793210
40,122,113,1,123,чувствовать,она,10.140821
42,145,105,6,146,подкараулить,кирпичный,-0.722384
53,160,124,2,163,успеть,тревога,5.222582


In [2]:
from tg.grammar_ru.algorithms import AntecedentCandidatesAlgorithm

alg = AntecedentCandidatesAlgorithm(reader.get_bundles(), 7)

c:\users\alexandra\desktop\grammar_ru\venv\lib\site-packages\slovnet\model\emb.py:46: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_numpy.cpp:178.)
  torch.from_numpy(navec.pq.indexes),


In [8]:
dbs = [x for x in reader.get_bundles()]
len(dbs)

14

In [24]:
with pd.option_context('display.max_rows', None, 'expand_frame_repr', False):
    print(alg.run_full(dbs[3]).drop(['pronoun_parent_id'], axis=1))

     pronoun_word_id  candidate_word_id  candidate_distance pronoun_parent_norm candidate_word_norm    product
12             39944              39932                   1           уговорить                  он   5.260468
16             40084              39991                   4              успеть                 дом   4.988981
23             40115              40092                   4             удаться             погожий   6.584331
41             40212              40200                   0           добраться              колдун   5.779660
47             40357              40330                   1           протянуть             мужчина   3.445329
55             40390              40384                   0            напугать                 раз   3.938898
60             40413              40393                   2              желать             мужчина   6.372273
63             40511              40421                   6          посмотреть                 зал   9.561722
7